In [1]:
#Dependencies 
import pandas as pd 
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import requests
import json
import time
import csv

In [2]:
# File to Load
import csv
load_file = "mlb_ticket_prices.csv"
mlb_ticket_prices = pd.read_csv(load_file, encoding= 'unicode_escape')
mlb_ticket_prices.head()

,Team,Average Ticket Price,Lat,Long
0,Arizona Diamondbacks,$50.00,33.452922,-112.038669
1,Atlanta Braves,$37.00,33.746910,-84.391239
2,Baltimore Orioles,$46.00,39.285243,-76.620103
3,Boston Red Sox,$70.00,42.346613,-71.098817
4,Chicago Cubs,$65.00,41.947201,-87.656413


In [3]:
## create engine
engine = create_engine('sqlite:///sports.db', echo = True)
sqlite_connection = engine.connect()

2020-10-18 11:12:58,219 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-18 11:12:58,222 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 11:12:58,223 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-18 11:12:58,224 INFO sqlalchemy.engine.base.Engine ()


In [74]:
Base=automap_base()
Base.prepare(sqlite_connection, reflect=True)
Base.classes.keys()

2020-10-17 16:28:59,748 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-10-17 16:28:59,750 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:28:59,752 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("nba_ticket_price")
2020-10-17 16:28:59,753 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:28:59,756 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'nba_ticket_price' AND type = 'table'
2020-10-17 16:28:59,757 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:28:59,759 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("nba_ticket_price")
2020-10-17 16:28:59,760 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:28:59,761 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("nba_ticket_price")
2020-10-17 16:28:59,762 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:28:59,763 INFO sqlalchemy.engine

['nba_ticket_price', 'nfl_ticket_price']

In [5]:
#Set name of table to create
mlb_tickets = "mlb_ticket_prices"
mlb_ticket_prices.to_sql(mlb_tickets, sqlite_connection, if_exists = 'fail')

2020-10-18 11:13:23,892 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("mlb_ticket_prices")
2020-10-18 11:13:23,893 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 11:13:23,895 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("mlb_ticket_prices")
2020-10-18 11:13:23,896 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 11:13:23,898 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE mlb_ticket_prices (
	"index" BIGINT, 
	"Team" TEXT, 
	"Average Ticket Price " TEXT, 
	"Lat" FLOAT, 
	"Long" FLOAT
)


2020-10-18 11:13:23,899 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 11:13:23,905 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-18 11:13:23,907 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_mlb_ticket_prices_index ON mlb_ticket_prices ("index")
2020-10-18 11:13:23,908 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 11:13:23,913 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-18 11:13:23,917 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-18 11:13:

In [6]:
#Putting primary key into ticket price table
create_query_sql='create table mlb_ticket_price (\
id_num bigint primary key,\
team text,\
ave tic price bigint,\
arena text,\
lat double,\
long double\
);'
sqlite_connection.execute(create_query_sql)

2020-10-18 11:13:26,216 INFO sqlalchemy.engine.base.Engine create table mlb_ticket_price (id_num bigint primary key,team text,ave tic price bigint,arena text,lat double,long double);
2020-10-18 11:13:26,217 INFO sqlalchemy.engine.base.Engine ()
2020-10-18 11:13:26,218 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) table mlb_ticket_price already exists
[SQL: create table mlb_ticket_price (id_num bigint primary key,team text,ave tic price bigint,arena text,lat double,long double);]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [53]:
sqlite_connection.execute('insert into mlb_ticket_price select * from mlb_ticket_prices')

2020-10-17 16:19:38,794 INFO sqlalchemy.engine.base.Engine insert into mlb_ticket_price select * from mlb_ticket_prices
2020-10-17 16:19:38,795 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:19:38,796 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: mlb_ticket_prices
[SQL: insert into mlb_ticket_price select * from mlb_ticket_prices]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [77]:
#Make fetch happen, check work 
sqlite_connection.execute('select * from mlb_ticket_prices').fetchall()

2020-10-17 16:30:06,649 INFO sqlalchemy.engine.base.Engine select * from mlb_ticket_prices
2020-10-17 16:30:06,651 INFO sqlalchemy.engine.base.Engine ()


[(0, 'Arizona Diamondbacks', '$50.00 ', 33.452921999999994, -112.038669),
 (1, 'Atlanta Braves', '$37.00 ', 33.74691, -84.391239),
 (2, 'Baltimore Orioles', '$46.00 ', 39.285243, -76.620103),
 (3, 'Boston Red Sox', '$70.00 ', 42.346613, -71.09881700000001),
 (4, 'Chicago Cubs', '$65.00 ', 41.947201, -87.656413),
 (5, 'Chicago White Sox', '$30.00 ', 41.830883, -87.635083),
 (6, 'Cincinnati Reds', '$47.00 ', 39.107183, -84.507713),
 (7, 'Cleveland Indians', '$62.00 ', 41.495149, -81.68709),
 (8, 'Colorado Rockies', '$39.00 ', 39.75698, -104.965329),
 (9, 'Detroit Tigers', '$38.00 ', 42.346354, -83.059619),
 (10, 'Houston Astros', '$56.00 ', 29.76045, -95.369784),
 (11, 'Kansas City Royals', '$47.00 ', 39.10222, -94.583559),
 (12, 'Los Angeles Angels', '$49.00 ', 33.799572, -117.889031),
 (13, 'Los Angeles Dodgers', '$46.00 ', 34.072437, -118.246879),
 (14, 'Miami Marlins', '$28.00 ', 25.954428, -80.238164),
 (15, 'Milwaukee Brewers', '$39.00 ', 43.04205, -87.905599),
 (16, 'Minnesota Twi